In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import time

ModuleNotFoundError: No module named 'numpy'

In [133]:
def calc_pcap(x):
    mu = np.asarray([4,4])
    var = np.asarray([[1, 0.8], [0.8, 1]])
    expo = np.exp(-0.5*(x-mu).T @ np.linalg.inv(var) @ (x-mu))
    return expo

In [162]:
np.random.seed(7)
S = 10000 # samples needed
Si2 = [0.01, 1, 100]

gridx, gridy = np.mgrid[2:7:0.1, 2:7:0.1]
pos = np.empty(gridx.shape + (2,))
pos[:, :, 0] = gridx; pos[:, :, 1] = gridy
p = st.multivariate_normal(mean = [4,4], cov = [[1, 0.8], [0.8, 1]])


for si2 in Si2:
    samples = np.empty([S, 2])
    current = np.asarray([0,0]) # z_0 
    var = si2*np.eye(2)
    print("si2 = ", si2)
    s = 0 # no of accepted samples so far
    stime = time.time()
    T = 0
    
    while s < S:
        T += 1
        
        samples[s] = current
        z = st.multivariate_normal.rvs(mean = current, cov = var)
        num = calc_pcap(z) * st.multivariate_normal.pdf(current, mean = z, cov = var)
        den = calc_pcap(current) * st.multivariate_normal.pdf(z, mean = current, cov = var)
        acc_prob = num / den

        if st.uniform.rvs() < acc_prob:
            s += 1
            if s % 1000 == 0: print("Samples collected : ", s)
            current = z

        if (s == 100 or s == 1000 or s == 5000 or s == 10000):
            plt.figure(figsize=(9,9))

            # plot of samples (blue)
            scat = plt.plot(samples[:s, 0], samples[:s, 1], ".", label = "Samples", alpha = 0.6, zorder=1)

            # plot of p(z) (red) [original]
            plt.contour(gridx, gridy, p.pdf(pos), levels = 0.05, colors = ["red"], linewidths=4, zorder = 5)

            # plot of p_cap(z) (yellow) [approximated]
            curr_mean = np.mean(samples[:s], axis = 0)
            curr_var = np.cov(samples[:s], rowvar= False)
            p_star = st.multivariate_normal(mean = curr_mean, cov = curr_var)
            limxlow = round(np.min(samples[:s, 0]))-1; limylow = round(np.min(samples[:s, 1]))-1;
            limxhigh = round(np.max(samples[:s, 0]))+1; limyhigh = round(np.max(samples[:s, 1]))+1;
            gridx_c, gridy_c = np.mgrid[limxlow:limxhigh:0.1, limylow:limyhigh:0.1]
            pos_c = np.empty(gridx_c.shape + (2,))
            pos_c[:, :, 0] = gridx_c; pos_c[:, :, 1] = gridy_c
            plt.contour(gridx_c, gridy_c, p_star.pdf(pos_c), levels = 0.05, colors = ["yellow"], linewidths = 4, zorder = 10)

            plt.title("Approximation with " + str(s) + " Samples for si2 = "+str(si2))
            plt.xlabel("x")
            plt.ylabel("y")
            proxy = [plt.Rectangle((0,0),0.01,1,fc = "red"), plt.Rectangle((0,0),0.01,1,fc = "yellow")]
            plt.legend(proxy, ["p(z)", "p_cap(z)"])
            plt.savefig("MH"+str(s)+"si2"+str(si2)+".png")
            plt.close() 
    
    print("Time for collecting 10000 samples when si2 = ", si2, "is", time.time()-stime)
    print("Rejection rate for si2 =", si2,"is", (T - s)/T)

si2 =  0.01
Samples collected :  1000
Samples collected :  2000
Samples collected :  3000
Samples collected :  4000
Samples collected :  5000
Samples collected :  6000
Samples collected :  7000
Samples collected :  8000
Samples collected :  9000
Samples collected :  10000
Time for collecting 10000 samples when si2 =  0.01 is 29.573988914489746
Rejection rate for si2 = 0.01 is 0.07978282874758442
si2 =  1
Samples collected :  1000
Samples collected :  2000
Samples collected :  3000
Samples collected :  4000
Samples collected :  5000
Samples collected :  6000
Samples collected :  7000
Samples collected :  8000
Samples collected :  9000
Samples collected :  10000
Time for collecting 10000 samples when si2 =  1 is 85.12750387191772
Rejection rate for si2 = 1 is 0.5936611133685493
si2 =  100
Samples collected :  1000
Samples collected :  2000
Samples collected :  3000
Samples collected :  4000
Samples collected :  5000
Samples collected :  6000
Samples collected :  7000
Samples collected : 